지연 속성에는 __getattr__, __getattribute__, __setattr__ 을 사용하자


클래스에 __getattr__ 메서드를 정의하면 객체의 인스턴스 딕셔너리에서 속성을 찾을 수 없을 때마다 이 메서드가 호출된다.

In [1]:

class LazyDB(object):
    def __init__(self):
        self.exists = 5

    def __getattr__(self, name):
        value = 'Value for %s' % name
        setattr(self, name, value)
        return value



In [2]:

# Example 2
data = LazyDB()
# exists 속성을 가지고 있다. 
print('Before:', data.__dict__)
print('foo:   ', data.foo)
print('After: ', data.__dict__)


Before: {'exists': 5}
foo:    Value for foo
After:  {'exists': 5, 'foo': 'Value for foo'}


LazyDB 의 "foo" 라는 속성을 접근했을 때 "setattr" foo 속성을 선언한다. 

In [23]:
class LoggingLazyDB(LazyDB):
        
    def __getattr__(self, name):
        print('Called __getattr__(%s)' % name)
        return super().__getattr__(name)


In [24]:

data = LoggingLazyDB()
print('exists:', data.exists)
print('foo:   ', data.foo)
print('foo:   ', data.foo)

exists: 5
Called __getattr__(foo)
foo:    Value for foo
foo:    Value for foo


exist 는 속성이 딕셔너리에 있으므로 절대 __getattr__ 이 호출되지 않는다.

foo 는 최초에는 호출되어서 foo 속성을 딕셔너리에 저장하였기 때문에

두번 이상 부터는 호출되지 않는다.


In [25]:
class ValidatingDB(object):
    def __init__(self):
        self.exists = 5

    def __getattribute__(self, name):
        print('Called __getattribute__(%s)' % name)
        try:
            return super().__getattribute__(name)
        except AttributeError:
            value = 'Value for %s' % name
            setattr(self, name, value)
            return value


In [26]:
data = ValidatingDB()
print('exists:', data.exists)
print('foo:   ', data.foo)
print('foo:   ', data.foo)

Called __getattribute__(exists)
exists: 5
Called __getattribute__(foo)
foo:    Value for foo
Called __getattribute__(foo)
foo:    Value for foo


파이썬에서는 "__getattribute__" 라는 또 다른 후크가 존재하며, 
이 메서드는 객체의 속성에 접근할때마다 호출되고, 해당 속성이 존재하더라도 역시 호출된다.

In [27]:
data = LoggingLazyDB()
print('Before:     ', data.__dict__)
print('foo exists: ', hasattr(data, 'foo'))
print('After:      ', data.__dict__)
print('foo exists: ', hasattr(data, 'foo'))

Before:      {'exists': 5}
Called __getattr__(foo)
foo exists:  True
After:       {'exists': 5, 'foo': 'Value for foo'}
foo exists:  True


종종 내장함수 hasattr 프로퍼티로 값이 있는지 확인하고 getattr 로 프로퍼티 값을 가져온다.

hasattr -> getattribute -> getattr -> setattr 순으로 호출된다.

In [29]:
data = ValidatingDB()
print('foo exists: ', hasattr(data, 'foo'))
print('foo exists: ', hasattr(data, 'foo'))

Called __getattribute__(foo)
foo exists:  True
Called __getattribute__(foo)
foo exists:  True


__getattribute__ 는 hasattr 로 호출할 때 마다 호출된다.  

In [30]:
class SavingDB(object):
    def __setattr__(self, name, value):
        # Save some data to the DB log
        super().__setattr__(name, value)

In [31]:
class LoggingSavingDB(SavingDB):
    def __setattr__(self, name, value):
        print('Called __setattr__(%s, %r)' % (name, value))
        super().__setattr__(name, value)

In [32]:

data = LoggingSavingDB()
print('Before: ', data.__dict__)
data.foo = 5
print('After:  ', data.__dict__)
data.foo = 7
print('Finally:', data.__dict__)

Before:  {}
Called __setattr__(foo, 5)
After:   {'foo': 5}
Called __setattr__(foo, 7)
Finally: {'foo': 7}


setattr 은 속성에 값을 할당할때 마다 호출된다.

In [33]:
class BrokenDictionaryDB(object):
    def __init__(self, data):
        self._data = data

    def __getattribute__(self, name):
        print('Called __getattribute__(%s)' % name)
        return self._data[name]


In [35]:
# stack over flow
# try:
#     data = BrokenDictionaryDB({'foo': 3})
#     data.foo
# except:
#     logging.exception('Expected')
# else:
#     assert False

__getattribute__ 를 잘못 사용하면 스택오버플로우가 발생할 수 있는 문제점이 있다.

__getattribute__ 안에서 self.data 에 접근하면 __getattribute__ 다시 호출되어 진다.




In [36]:
class DictionaryDB(object):
    def __init__(self, data):
        self._data = data

    def __getattribute__(self, name):
        data_dict = super().__getattribute__('_data')
        return data_dict[name]

data = DictionaryDB({'foo': 3})
print(data.foo)

3


super().__getattribute__('_data') 로 접근하여 이 문제를 피할 수 있다. 

In [38]:
# class DictionaryDB(object):
#     def __init__(self, data):
#         self._data = data
#     def __setattr__(self, key, value):
#         self.data = value
# 
# data = DictionaryDB({'foo': 3})
# print(data.foo)

마찬가지로 settattr 에서도 super.__setattr__() 을 호출해야 스택오버플로우를 피할 수 있다.